In [1]:
import sys
!{sys.executable} -m pip install optuna

In [2]:
import sys
sys.path.append('../utils/')
sys.path.append('../paviaUTools/')
# sys.path.insert(1, '../utils')
# sys.path.insert(2, '../paviaUTools')

import matplotlib.pyplot as plt
from datasetLoader import datasetLoader
import os
import numpy as np
from whole_pipeline import whole_pipeline_all, whole_pipeline_divided, whole_pipeline_divided_parallel
import torch
from plots import *
from weights_anal import *
from MetaLearner import HDDOnBands
import consts

import gc

torch.cuda.empty_cache()
gc.collect()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

parent_dir = os.path.join(os.getcwd(),"..")
csv_path = os.path.join(parent_dir, 'datasets', 'paviaU.csv')
gt_path = os.path.join(parent_dir, 'datasets', 'paviaU_gt.csv')
# csv_path = os.path.join(parent_dir, 'datasets', 'pavia.csv')
# gt_path = os.path.join(parent_dir, 'datasets', 'pavia_gt.csv')

dsl = datasetLoader(csv_path, gt_path)

df = dsl.read_dataset(gt=False)
X = np.array(df)
X = X.reshape((610,340, 103))
# X = X.reshape((1096, 715, 102))

df = dsl.read_dataset(gt=True)
y = np.array(df)

X = torch.from_numpy(X)
y = torch.from_numpy(y)

X = X.to(device)
y = y.to(device)

In [3]:
factor = 7
reps = 5
min_hyperparam = 0.1
max_hyperparam = 10

In [4]:
import optuna

def evaluate(c):
    torch.cuda.empty_cache()
    gc.collect()

    consts.CONST_C = c

    avg_acc_test = 0.0
    for _ in range(reps):
        train_acc,test_acc, test_preds,test_gt = whole_pipeline_all(X,y, factor, factor, is_normalize_each_band=True, method_label_patch='most_common')
        avg_acc_test += test_acc/reps

    score = avg_acc_test
    return score

# Define the objective function for Optuna
def objective(trial):
    # Suggest a value for the hyperparameter within a given range
    hyperparameter = trial.suggest_float('c', min_hyperparam, max_hyperparam)
    
    # Evaluate the hyperparameter
    score = evaluate(hyperparameter)
    
    return score


In [5]:
# Create a study object and specify the direction of optimization
study = optuna.create_study(direction='maximize')  # or 'maximize' depending on your evaluation function

# Start the optimization
study.optimize(objective, n_trials=100)

# Print the best hyperparameter and its corresponding score
print("Best Hyperparameter: ", study.best_params['c'])
print("Best Score: ", study.best_value)


[I 2024-06-19 12:00:26,562] A new study created in memory with name: no-name-e1253c35-9506-48f4-9e97-2eecce375d32


XXXXXXX IN METHOD XXXXXXXXX
WHOLE METHOD TIME:  1.33872652053833
XXXXXXX IN CLASSIFICATION XXXXXXXXX
total classified:  36108
total classified:  6666
Train Accuracy:  0.9296554780104131
Test Accuracy:  0.5258025802580258
XXXXXXX IN METHOD XXXXXXXXX
WHOLE METHOD TIME:  1.2226407527923584
XXXXXXX IN CLASSIFICATION XXXXXXXXX
total classified:  33868
total classified:  8906
Train Accuracy:  0.9263021140899964
Test Accuracy:  0.6648326970581631
XXXXXXX IN METHOD XXXXXXXXX
WHOLE METHOD TIME:  0.9181597232818604
XXXXXXX IN CLASSIFICATION XXXXXXXXX
total classified:  33508
total classified:  9266
Train Accuracy:  0.9282260952608332
Test Accuracy:  0.6772069933088711
XXXXXXX IN METHOD XXXXXXXXX
WHOLE METHOD TIME:  1.108297348022461
XXXXXXX IN CLASSIFICATION XXXXXXXXX
total classified:  34695
total classified:  8079
Train Accuracy:  0.9252053610030264
Test Accuracy:  0.5744522836984776
XXXXXXX IN METHOD XXXXXXXXX
